In [1]:
import pandas as pd
import numpy as np
df1=pd.read_csv('F:/0Sem 7/ML Lab/amazon food review dataset/Reviews.csv')
df1.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [2]:
score=df1.values[:,6]
text=df1.values[:,9]
reviews=np.vstack((score,text)).T
print(score.shape, text.shape, reviews.shape)

(568454,) (568454,) (568454, 2)


In [3]:
p=0
n=0
for i in range(reviews.shape[0]):
    if reviews[i,0] > 3:
        reviews[i,0]=0 #positive review
        p=p+1
    else:
        reviews[i,0]=1 #negative review
        n=n+1

In [4]:
reviews = reviews[reviews[:,0].argsort()]   #sort by 1st column

In [5]:
train=[]
for i in range(5000):
    train.append(reviews[i])
    
for i in range(443777,443777+5000):
    train.append(reviews[i])
    
train=np.asarray(train)
train.shape

(10000, 2)

In [6]:
import random as r
test=[]
for i in range(2000):
    index=r.randint(0,reviews.shape[0])
    test.append(reviews[index])
    
test=np.asarray(test)
test.shape

(2000, 2)

In [7]:
train_all_words=[]
for i in range(train.shape[0]):
    train_all_words.append(train[i,1].split())
    
train_all_words = [item for sublist in train_all_words for item in sublist]

test_all_words=[]
for i in range(test.shape[0]):
    test_all_words.append(test[i,1].split())
    
test_all_words = [item for sublist in test_all_words for item in sublist]

from collections import Counter
def common_words(words, number_of_words, reverse=False):
    counter = Counter(words)
    return sorted(counter, key = counter.get, reverse=reverse)[:number_of_words]

train_least_common=common_words(train_all_words,200)
train_most_common=common_words(train_all_words,200,reverse=True)

test_least_common=common_words(test_all_words,200)
test_most_common=common_words(test_all_words,200,reverse=True)

In [8]:
for i in range(train.shape[0]):
    train[i,1]=train[i,1].split()
    
for i in range(test.shape[0]):
    test[i,1]=test[i,1].split()

In [9]:
for i in range(train.shape[0]):
    for item in train_most_common:
        if item in train[i,1]:
            train[i,1].remove(item)
    for item in train_least_common:
        if item in train[i,1]:
            train[i,1].remove(item)
            
for i in range(test.shape[0]):
    for item in test_most_common:
        if item in test[i,1]:
            test[i,1].remove(item)
    for item in test_least_common:
        if item in test[i,1]:
            test[i,1].remove(item)

In [10]:
for i in range(train.shape[0]):
    train[i,1]=" ".join(train[i,1])
    
for i in range(test.shape[0]):
    test[i,1]=" ".join(test[i,1])

In [11]:
train_docs=[]
test_docs=[]
train_label=[]
test_label=[]
for i in range(train.shape[0]):
    train_docs.append(train[i,1])
    train_label.append(train[i,0])
    
for i in range(test.shape[0]):
    test_docs.append(test[i,1])
    test_label.append(test[i,0])

In [12]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(train_docs)
vocab_size = len(t.word_index) + 1
#integer encode the documents
encoded_docs = t.texts_to_sequences(train_docs)

#print(encoded_docs)
#pad documents to a max length of 4 words

max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

#print(padded_docs)
#load the whole embedding into memory

embeddings_index = dict()
f = open('F:/0Sem 7/ML Lab/glove/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

#print('Loaded %s word vectors.' % len(embeddings_index))

#create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Using TensorFlow backend.


In [13]:
embedding_matrix.shape

(19814, 100)

In [14]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#define model
model = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model.add(e)
model.add(layers.Conv1D(32,4, activation='relu'))
model.add(layers.Dropout(rate=0.8))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(64,3,activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [15]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1981400   
_________________________________________________________________
conv1d (Conv1D)              (None, 97, 32)            12832     
_________________________________________________________________
dropout (Dropout)            (None, 97, 32)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 48, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 64)            6208      
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 64)            256       
_________________________________________________________________
flatten (Flatten)            (None, 2944)              0

In [16]:
for layer in model.layers:
    print(layer.output_shape)

(None, 100, 100)
(None, 97, 32)
(None, 97, 32)
(None, 48, 32)
(None, 46, 64)
(None, 46, 64)
(None, 2944)
(None, 128)
(None, 1)


In [17]:
train_label=np.asarray(train_label)
train_label.shape

(10000,)

In [18]:
padded_docs.shape

(10000, 100)

In [19]:
# prepare tokenizer
t1 = Tokenizer()
t1.fit_on_texts(test_docs)
vocab_size1 = len(t1.word_index) + 1
#integer encode the documents
encoded_docs1 = t1.texts_to_sequences(test_docs)

#print(encoded_docs)
#pad documents to a max length of 4 words

max_length1 = 100
padded_docs1 = pad_sequences(encoded_docs1, maxlen=max_length1, padding='post')
test_label=np.asarray(test_label)
#print(padded_docs)

In [20]:
# fit the model
model.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=50, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/50
10000/10000 [==============================] - 15s 1ms/sample - loss: 0.7518 - accuracy: 0.5383 - val_loss: 0.5890 - val_accuracy: 0.6545
Epoch 2/50
10000/10000 [==============================] - 4s 411us/sample - loss: 0.6280 - accuracy: 0.6508 - val_loss: 0.5836 - val_accuracy: 0.7090
Epoch 3/50
10000/10000 [==============================] - 4s 396us/sample - loss: 0.5709 - accuracy: 0.7029 - val_loss: 0.8598 - val_accuracy: 0.4430
Epoch 4/50
10000/10000 [==============================] - 4s 409us/sample - loss: 0.5436 - accuracy: 0.7237 - val_loss: 0.7173 - val_accuracy: 0.5605
Epoch 5/50
10000/10000 [==============================] - 4s 406us/sample - loss: 0.5264 - accuracy: 0.7391 - val_loss: 0.9473 - val_accuracy: 0.4310
Epoch 6/50
10000/10000 [==============================] - 4s 398us/sample - loss: 0.5079 - accuracy: 0.7503 - val_loss: 0.7458 - val_accuracy: 0.5360
Epoch 7/50
10000/10000 [============================